In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
from scipy import interpolate
from ccfj import GetStationPairs
import pandas as pd
import yaml
import geopandas as gp
import openpyxl
from openpyxl import load_workbook
from concurrent.futures import ThreadPoolExecutor

from scipy.interpolate import make_interp_spline
from scipy.stats import ks_2samp
from scipy.spatial import ConvexHull
from skimage import measure

In [2]:
import sys
sys.path.append(r'../tools_F-J/')
from toollib_standard import plotlib

In [3]:
name_project = 'project/output_FJSJ_17-02/'               # Harmon server
name_project

'project/output_FJSJ_17-02/'

In [4]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

dir_CC_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project:  /shdisk/rem2/Harmon/F-J/San/project/output_FJSJ_17-02/


In [5]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [6]:
key_subworks = info_basic['key_subworks']
key_subworks

['07-03',
 '08-03',
 '09-03',
 '10-03',
 '11-03',
 '12-03',
 '13-03',
 '14-03',
 '15-03',
 '16-03',
 '17-03',
 '18-03',
 '19-03',
 '20-03',
 '21-03',
 '22-03',
 '23-03',
 '24-03',
 '25-03',
 '26-03',
 '27-03',
 '28-03',
 '29-03',
 '30-03',
 '31-03',
 '32-03',
 '33-03',
 '34-03',
 '35-03',
 '36-03',
 '37-03',
 '38-03',
 '39-03',
 '40-03',
 '41-03',
 '42-03',
 '43-03',
 '44-03',
 '45-03',
 '46-03',
 '47-03',
 '48-03',
 '49-03',
 '50-03',
 '51-03',
 '52-03',
 '53-03',
 '54-03',
 '07-04',
 '08-04',
 '09-04',
 '10-04',
 '11-04',
 '12-04',
 '13-04',
 '14-04',
 '15-04',
 '16-04',
 '17-04',
 '18-04',
 '19-04',
 '20-04',
 '21-04',
 '22-04',
 '23-04',
 '24-04',
 '25-04',
 '26-04',
 '27-04',
 '28-04',
 '29-04',
 '30-04',
 '31-04',
 '32-04',
 '33-04',
 '34-04',
 '35-04',
 '36-04',
 '37-04',
 '38-04',
 '39-04',
 '40-04',
 '41-04',
 '42-04',
 '43-04',
 '44-04',
 '45-04',
 '46-04',
 '47-04',
 '48-04',
 '49-04',
 '50-04',
 '51-04',
 '52-04',
 '53-04',
 '54-04',
 '07-05',
 '08-05',
 '09-05',
 '10-05',


In [7]:
dir_image = dir_project+info_basic['dir_image']
dir_ds = dir_project+info_basic['dir_ds']
print('dir_image: ', dir_image)
print('dir_ds: ', dir_ds)

dir_image:  /shdisk/rem2/Harmon/F-J/San/project/output_FJSJ_17-02/image_17-02/
dir_ds:  /shdisk/rem2/Harmon/F-J/San/project/output_FJSJ_17-02/ds_17-02/


In [8]:
dir_disp =  'autopick'+info_basic['tag']+'/'
info_basic['dir_disp'] = dir_disp
dir_disp = dir_project+dir_disp
if not os.path.exists(dir_disp):
    os.makedirs(dir_disp)
print('dir_disp: ', dir_disp)

dir_disp:  /shdisk/rem2/Harmon/F-J/San/project/output_FJSJ_17-02/autopick17-02//


In [9]:
ds_remove_all = {}
for key_subwork in key_subworks:
    ds = h5py.File(dir_ds+'ds_'+key_subwork+'.h5', 'r')
    ds_remove = ds['ds_remove'][0]
    ds.close()
    ds_remove_all[key_subwork] = ds_remove

### Pick probes' Phase velocity automatically

In [10]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

In [11]:
def cluster_fundamental(ds):
    global f
    global c
    global f_new
    global c_new
    global dir_ds
    global threshold
    # 归一化
    #ds = ds / np.max(ds)  
    ds = np.nan_to_num(ds)
    for i in range(ds.shape[1]):
        ds[:,i] = ds[:,i]/np.max(ds[:,i])

    binary_ds = ds > threshold
    label_ds,num_labels = measure.label(binary_ds, connectivity=2, return_num=True)
    region_areas = [np.sum(label_ds==i) for i in range(1,num_labels+1)]
    if num_labels == 0:
        return np.zeros(0),np.zeros(0)
    max_area_label = np.argmax(region_areas) + 1
    largest_cluster = label_ds == max_area_label
    indices = np.argwhere(largest_cluster)

    ff = f_new[indices[:,1]]
    cc = c_new[indices[:,0]]

    return ff,cc

In [12]:
def pick_fundamental(key):
    global f
    global c
    global f_new
    global c_new
    global flag_f
    global flag_c
    global dir_ds
    global threshold
    global dir_disp
    global dir_ds
    global info_basic
    global info_basic_bi
    global smooth_cluster
    global smooth_pick
    global key_subworks
    global ds_remove_all

    
    ds_remove = ds_remove_all[key]
    grace = 0.0

    f_start = flag_f[0]
    f_end = flag_f[1]
    c_min = flag_c[0]
    c_max = flag_c[1]

    ff_all = []
    cc_all = []


    while f_start != f_end:
        #print(f_start,f_end)
        f_new = f[np.logical_and(f>f_start,f <= f_end)]
        c_new = c[np.logical_and(c>c_min,c < c_max)]
        ds_new0 = ds_remove[np.logical_and(c>c_min,c < c_max),:]
        ds_new = ds_new0[:,np.logical_and(f>f_start,f <= f_end)]
        # 对数组进行横向的平滑
        ds_new = pd.DataFrame(ds_new)
        #ds_new = ds_new.rolling(100).mean()
        ds_new = ds_new.T
        ds_new = ds_new.rolling(smooth_cluster).mean()
        ds_new = np.array(ds_new)
        ds_new = ds_new.T
        ff,cc = cluster_fundamental(ds_new)
        if len(ff_all) == 0:
            ff_all = ff
            cc_all = cc
        elif len(ff) != 0:
            ff_all = np.hstack((ff_all,ff))
            cc_all = np.hstack((cc_all,cc))
        else:
            #print('this')
            break
        f_start = np.max(ff)
        c_max = np.max(cc[ff==f_start]) + grace
    
    # smooth
    # 转置
    ds_smooth = ds_remove.T
    ds_smooth = pd.DataFrame(ds_smooth)
    ds_smooth = ds_smooth.rolling(smooth_pick).mean()
    ds_smooth = np.array(ds_smooth)
    ds_smooth = ds_smooth.T
    ds_smooth = pd.DataFrame(ds_smooth)
    ds_smooth = ds_smooth.rolling(smooth_pick).mean()
    ds_smooth = np.array(ds_smooth)

    f_range = set(ff_all)
    f_range = list(f_range)
    c_pick = []
    for i in range(len(f_range)):
        f_this = f_range[i]
        c_this = cc_all[ff_all==f_this]
        c_range = [np.min(c_this),np.max(c_this)]
        c_new = c[np.logical_and(c>=c_range[0],c<=c_range[1])]
        c_pick.append(c_new[np.argmax(ds_smooth[np.logical_and(c>=c_range[0],c<=c_range[1]),f==f_this])])
    
    # save as txt
    np.savetxt(dir_disp+'autopick_'+key+'.txt', np.array([f_range,c_pick]).T, fmt='%.4f', delimiter=',')
    print('finish '+ key + ' '+str(key_subworks.index(key)+1) + '/'+str(len(key_subworks)))

In [13]:
f = info_basic_bi['f']
c = np.linspace(info_basic['c_min'],info_basic['c_max'],info_basic['c_num'])
flag_c = [0.2,1.2]
flag_f = [2,15]
threshold = 0.3
f_new = f[np.logical_and(f>flag_f[0],f < flag_f[1])]
c_new = c[np.logical_and(c>flag_c[0],c < flag_c[1])]
smooth_cluster = 50
smooth_pick = 5

In [14]:
for key_subwork in key_subworks:
    pick_fundamental(key_subwork)

/tmp/ipykernel_29545/2543580824.py:12: RuntimeWarning: invalid value encountered in divide
  ds[:,i] = ds[:,i]/np.max(ds[:,i])


finish 07-03 1/768
finish 08-03 2/768


/tmp/ipykernel_29545/2543580824.py:12: RuntimeWarning: invalid value encountered in divide
  ds[:,i] = ds[:,i]/np.max(ds[:,i])
/tmp/ipykernel_29545/2543580824.py:12: RuntimeWarning: invalid value encountered in divide
  ds[:,i] = ds[:,i]/np.max(ds[:,i])


finish 09-03 3/768
finish 10-03 4/768


/tmp/ipykernel_29545/2543580824.py:12: RuntimeWarning: invalid value encountered in divide
  ds[:,i] = ds[:,i]/np.max(ds[:,i])
/tmp/ipykernel_29545/2543580824.py:12: RuntimeWarning: invalid value encountered in divide
  ds[:,i] = ds[:,i]/np.max(ds[:,i])


...

/tmp/ipykernel_29545/2543580824.py:12: RuntimeWarning: invalid value encountered in divide
  ds[:,i] = ds[:,i]/np.max(ds[:,i])
/tmp/ipykernel_29545/2543580824.py:12: RuntimeWarning: invalid value encountered in divide
  ds[:,i] = ds[:,i]/np.max(ds[:,i])


finish 51-18 765/768
finish 52-18 766/768


/tmp/ipykernel_29545/2543580824.py:12: RuntimeWarning: invalid value encountered in divide
  ds[:,i] = ds[:,i]/np.max(ds[:,i])
/tmp/ipykernel_29545/2543580824.py:12: RuntimeWarning: invalid value encountered in divide
  ds[:,i] = ds[:,i]/np.max(ds[:,i])


finish 53-18 767/768
finish 54-18 768/768


/tmp/ipykernel_29545/2543580824.py:12: RuntimeWarning: invalid value encountered in divide
  ds[:,i] = ds[:,i]/np.max(ds[:,i])


In [15]:
"""
nThreads = 60
pool = ThreadPoolExecutor(max_workers = nThreads)
for key_subwork in key_subworks:
    pool.submit(pick_fundamental,key_subwork)
pool.shutdown()
"""

'\nnThreads = 60\npool = ThreadPoolExecutor(max_workers = nThreads)\nfor key_subwork in key_subworks:\n    pool.submit(pick_fundamental,key_subwork)\npool.shutdown()\n'

In [16]:
def plot_disp(key_subwork):
    global dir_disp
    global f
    global c
    global dir_image_disp

    fig,ax = plt.subplots(1,1,figsize=(8,6))
    dir_ds = dir_project + info_basic['dir_ds']
    ds = h5py.File(dir_ds+'ds_'+key_subwork+'.h5', 'r')
    ds_remove = ds['ds_remove'][0]

    title0 = key_subwork
    ax = plotlib.plot_fj(ax,ds_remove,title0,f,c,0,c_map='jet')

    f_pick = np.loadtxt(dir_disp+'autopick_'+key_subwork+'.txt',delimiter=',')[:,0]
    c_pick = np.loadtxt(dir_disp+'autopick_'+key_subwork+'.txt',delimiter=',')[:,1]
    ax.scatter(f_pick,c_pick,marker='o',s=10,color='k')
    plt.savefig(dir_image_disp+'autopick_'+key_subwork+'.png',dpi=60)
    plt.close()
    print('finish '+key_subwork)

In [17]:
dir_image_disp = 'image_autopick/'
if not os.path.exists(dir_project+dir_image_disp):
    os.makedirs(dir_project+dir_image_disp)
dir_image_disp = dir_project+dir_image_disp

In [18]:
for key_subwork in key_subworks:
    plot_disp(key_subwork)

finish 07-03
finish 08-03
finish 09-03
finish 10-03
finish 11-03
finish 12-03
finish 13-03
finish 14-03
finish 15-03
finish 16-03
finish 17-03
finish 18-03
finish 19-03
finish 20-03
finish 21-03
finish 22-03
finish 23-03
finish 24-03
finish 25-03
finish 26-03
finish 27-03
finish 28-03
finish 29-03
finish 30-03
finish 31-03
finish 32-03
finish 33-03
finish 34-03
finish 35-03
finish 36-03
finish 37-03
finish 38-03
finish 39-03
finish 40-03
finish 41-03
finish 42-03
finish 43-03
finish 44-03
finish 45-03
finish 46-03
finish 47-03
finish 48-03
finish 49-03
finish 50-03
finish 51-03
finish 52-03
finish 53-03
finish 54-03
finish 07-04
finish 08-04
finish 09-04
finish 10-04
finish 11-04
finish 12-04
finish 13-04
finish 14-04
finish 15-04
finish 16-04
finish 17-04
finish 18-04
finish 19-04
finish 20-04
finish 21-04
finish 22-04
finish 23-04
finish 24-04
finish 25-04
finish 26-04
finish 27-04
finish 28-04
finish 29-04
finish 30-04
finish 31-04
finish 32-04
finish 33-04
finish 34-04
finish 35-04

finish 46-05
finish 47-05
finish 48-05
finish 49-05
finish 50-05
finish 51-05
finish 52-05
finish 53-05
finish 54-05
finish 07-06
finish 08-06
finish 09-06
finish 10-06
finish 11-06
finish 12-06
finish 13-06
finish 14-06
finish 15-06
finish 16-06
finish 17-06
finish 18-06
finish 19-06
finish 20-06
finish 21-06
finish 22-06
finish 23-06
finish 24-06
finish 25-06
finish 26-06
finish 27-06
finish 28-06
finish 29-06
finish 30-06
finish 31-06
finish 32-06
finish 33-06
finish 34-06
finish 35-06
finish 36-06
finish 37-06
finish 38-06
finish 39-06
finish 40-06
finish 41-06
finish 42-06
finish 43-06
finish 44-06
finish 45-06
finish 46-06
finish 47-06
finish 48-06
finish 49-06
finish 50-06
finish 51-06
finish 52-06
finish 53-06
finish 54-06
finish 07-07
finish 08-07
finish 09-07
finish 10-07
finish 11-07
finish 12-07
finish 13-07
finish 14-07
finish 15-07
finish 16-07
finish 17-07
finish 18-07
finish 19-07
finish 20-07
finish 21-07
finish 22-07
finish 23-07
finish 24-07
finish 25-07
finish 26-07